In [143]:
import json
import os
from elasticsearch import Elasticsearch
import time
import datetime

In [62]:
es = Elasticsearch(['LINK_TO_ELASTIC'])

In [63]:
es.ping()

True

In [64]:
def send_json(content):
    for r in content['GraphImages']:
        location = ({} if r.get('location', {}) is None else r.get('location', {})).get('address_json', None)
        if location is not None:
            location = json.loads(location)
        doc = {
            'id': r['id'],
            'owner_id': r.get('owner_id', None),
            'location': location,
            'likes': r.get('edge_liked_by', {}).get('count', None),
            'taken_at_timestamp': r['taken_at_timestamp'] * 1000,
            'tags': r.get('tags', []),
            'edge_media_to_caption': r['edge_media_to_caption'],
            'edge_media_to_comment': {
                'count': r['edge_media_to_comment']['count'],
                'data': {
                    'text': [t['text'] for t in r['edge_media_to_comment'].get('data', [])]
                }
            }
        }

        es.index(index='instagram', body=doc, id=r['id'])

In [65]:
output = [dI for dI in os.listdir('scraper') if os.path.isdir(os.path.join('scraper',dI)) and dI != '.idea' and dI != 'venv']

In [66]:
for file in output:
    print(f'loading {file}')
    filename = 'scraper/' + file + '/' + file + '.json'
    with open(filename) as f:
        c = json.load(f)
        send_json(c)

loading feminismo
loading timesupnow
loading effyourbeautystandards
loading bumble
loading iwillgoout
loading prochoice
loading bringbackourgirls
loading whyididntreport
loading heforshe
loading womensreality
loading whyistayed
loading girlpower
loading iamanastywoman
loading makerswomen
loading yesallwomen
loading selflove
loading fem2
loading girlboss
loading feminist
loading believesurvivors
loading generationequality
loading gurlstalk
loading womensrights
loading intersectionalfeminism
loading feministart
loading women
loading sadgirlsclub
loading womensupportingwomen
loading equality
loading lgbt
loading womensmarch
loading ocasio2018
loading activist
loading strongwomen
loading womenempowerment
loading genderequality
loading humanrights
loading equalrights
loading womenshould
loading feminista
loading metoo
loading thefutureisfemale
loading tothegirls
loading feminism
loading iamwithher
loading niunamenos
loading believewomen
loading timesup
loading woman
loading nomoore


In [53]:
instagram_mapping = {
  "mappings": {
    "dynamic": False,
      "date_detection": False, 
    "properties": {
        "id": {
            "type": "keyword"
        },
        "owner_id": {
            "type": "keyword"
        },
        "location": {
            "properties": {
                "street_address": {
                    "type": "text",
                    "fields": {
                      "raw": { 
                        "analyzer": "english",
                        "index": True,
                        "search_analyzer": "english",
                        "type": "text"
                      }
                    }
                },
                "zip_code": {
                    "type": "text",
                    "fields": {
                      "raw": { 
                        "analyzer": "english",
                        "index": True,
                        "search_analyzer": "english",
                        "type": "text"
                      }
                    }
                },
                "city_name": {
                    "type": "text",
                    "fields": {
                      "raw": { 
                        "analyzer": "english",
                        "index": True,
                        "search_analyzer": "english",
                        "type": "text"
                      }
                    }
                },
                "region_name": {
                    "type": "text",
                    "fields": {
                      "raw": { 
                        "analyzer": "english",
                        "index": True,
                        "search_analyzer": "english",
                        "type": "text"
                      }
                    }
                },
                "country_code": {
                    "type": "text",
                    "fields": {
                      "raw": { 
                        "analyzer": "english",
                        "index": True,
                        "search_analyzer": "english",
                        "type": "text"
                      }
                    }
                },
                "exact_city_match": {
                    "type": "boolean"
                },
                "exact_region_match": {
                    "type": "boolean"
                },
                "exact_country_match": {
                    "type": "boolean"
                }
            }
        },
        "likes": {
            "type": "integer"
        },
        "taken_at_timestamp": {
              "type": "date"
        },
        "tags": {
            "type": "keyword",
        },
        "edge_media_to_caption": {
            "properties": {
                "edges": {
                    "properties": {
                        "node": {
                            "properties": {
                                "text": {
                                    "analyzer": "english",
                                    "index": True,
                                    "search_analyzer": "english",
                                    "type": "text"
                                }
                            }
                        }
                    }
                },
            },
        },
        "edge_media_to_comment": {
            "properties": {
                "count": {
                    "type": "integer"
                },
                "data": {
                    "properties": {
                        "text": {
                           
                                "analyzer": "english",
                                "index": True,
                                "search_analyzer": "english",
                                "type": "text"
                        }
                    }
                }
            }
        }
    }
  }
}

In [54]:
# es.indices.delete('instagram')

{'acknowledged': True}

In [55]:
response = es.indices.create(
    index="instagram",
    body=instagram_mapping,
    ignore=400 # ignore 400 already exists code
)

In [56]:
response

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'instagram'}

In [82]:
for r in content['GraphImages']:
    doc = {
        'id': r['id'],
        'taken_at_timestamp': r['taken_at_timestamp']*1000,
        'tags': r['tags'],
        'edge_media_to_caption': r['edge_media_to_caption'],
        'edge_media_to_comment': {
            'count': r['edge_media_to_comment']['count'],
            'data': {
                'text': [t['text'] for t in r['edge_media_to_comment']['data']]
            }
        }
    }
    
    es.index(index='instagram', body=doc, id=r['id'])

{'_index': 'instagram', '_type': '_doc', '_id': '2207630431050817969', '_version': 7, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 204, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2277798049709643340', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 205, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2263533628881143642', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 206, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2288925573747425943', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 207, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2263250417821803656', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 208, '_primary_term': 1}
{'_index':

{'_index': 'instagram', '_type': '_doc', '_id': '2304167125272557191', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 247, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2303946104905077389', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 248, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2303910839139171234', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 249, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2303896817269544747', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 250, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2303878591131193956', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 251, '_primary_term': 1}
{'_index':

{'_index': 'instagram', '_type': '_doc', '_id': '2302948425644449124', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 293, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2302940413392518401', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 294, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2302852355346666888', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 295, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2302851828030631795', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 296, '_primary_term': 1}
{'_index': 'instagram', '_type': '_doc', '_id': '2302842417060447095', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 2, 'failed': 0}, '_seq_no': 297, '_primary_term': 1}
{'_index':

In [119]:
reddit_mapping = {
    "mappings": {
        "dynamic": False,
        "date_detection": False,
        "properties": {
            "id": {
                "type": "keyword"
            },
            "title": {
                "type": "keyword"
            },
            "score": {
                "type": "integer"
            },
            "created_utc": {
                "type": "date"
            },
            "num_comments": {
                "type": "integer"
            },
            "author": {
                "type": "keyword"
            },
            "selftext": {
                "type": "keyword"
            },
            "comments": {
                "properties": {
                    "body": {
                        "analyzer": "english",
                        "index": True,
                        "search_analyzer": "english",
                        "type": "text"
                    },
                    "created_utc": {
                        "type": "date"
                    },
                    "id": {
                        "type": "keyword"
                    },
                    "is_submitter": {
                        "type": "boolean"
                    },
                    "link_id": {
                        "type": "keyword"
                    },
                    "parent_id": {
                        "type": "keyword"
                    },
                    "score": {
                        "type": "integer"
                    },
                    "subreddit_id": {
                        "type": "keyword"
                    },
                    "author": {
                        "properties": {
                            "name": {
                                "type": "keyword",
                            },
                            "id":  {
                                "type": "keyword",
                            }
                        }
                    },
                    "comments": {
                        "properties": {
                            "body": {
                                "analyzer": "english",
                                "index": True,
                                "search_analyzer": "english",
                                "type": "text"
                            },
                            "created_utc": {
                                "type": "date"
                            },
                            "id": {
                                "type": "keyword"
                            },
                            "is_submitter": {
                                "type": "boolean"
                            },
                            "link_id": {
                                "type": "keyword"
                            },
                            "parent_id": {
                                "type": "keyword"
                            },
                            "score": {
                                "type": "integer"
                            },
                            "subreddit_id": {
                                "type": "keyword"
                            },
                            "author": {
                                "properties": {
                                    "name": {
                                        "type": "keyword",
                                    },
                                    "id": {
                                        "type": "keyword",
                                    }
                                }
                            },
                            "comments": {
                                "properties": {
                                    "body": {
                                        "analyzer": "english",
                                        "index": True,
                                        "search_analyzer": "english",
                                        "type": "text"
                                    },
                                    "created_utc": {
                                        "type": "date"
                                    },
                                    "id": {
                                        "type": "keyword"
                                    },
                                    "is_submitter": {
                                        "type": "boolean"
                                    },
                                    "link_id": {
                                        "type": "keyword"
                                    },
                                    "parent_id": {
                                        "type": "keyword"
                                    },
                                    "score": {
                                        "type": "integer"
                                    },
                                    "subreddit_id": {
                                        "type": "keyword"
                                    },
                                    "author": {
                                        "properties": {
                                            "name": {
                                                "type": "keyword",
                                            },
                                            "id": {
                                                "type": "keyword",
                                            }
                                        }
                                    }
                                }
                            }
                        },

                    }
                },
            },
        }
    }
}

In [120]:
response = es.indices.create(
    index="reddit",
    body=reddit_mapping,
    ignore=400 # ignore 400 already exists code
)

In [121]:
response

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'reddit'}

In [122]:
def send_reddit_json(content):
    for r in content:
        r['url'] = r.get('url', '')
        del r['url']
        r['created_utc'] = int(r.get('created_utc', 0) * 1000)
        for c in r.get('comments', []):
            c['created_utc'] = int(c.get('created_utc', 0) * 1000)
            del c['is_submitter']
            del c['link_id']
            del c['parent_id']
            del c['subreddit_id']
            del c['comments']
#             for cc in c['comments']:
#                 if cc['id'] != c['id']:
#                     del cc['is_submitter']
#                     del cc['link_id']
#                     del cc['parent_id']
#                     del cc['subreddit_id']
#                     cc['created_utc'] = int(cc['created_utc'] * 1000)
#         print(len(json.dumps(r)))
        resp = es.index(index='reddit', body=r, id=r['id'])
#         print(f'Send {resp}')

In [123]:
reddit_output = [dI for dI in os.listdir('scraper') if not os.path.isdir(os.path.join('scraper',dI)) \
                 and not dI.startswith('twitter') and dI.endswith('json')]


In [125]:
reddit_output = ['TheGirlSurvivalGuide.json',
 'GirlsGoneWired.json',
 'feminism.json',
 'antifeminists.json',
 'femmethoughts.json',
 'GirlGamers.json',
 'scientits.json',
 'BreadTube.json',
 'blackladies.json',
 'NotHowGirlsWork.json',
 'ContraPoints.json',
 'FemaleDatingStrategy.json',
 'women.json',
 'WitchesVsPatriarchy.json']

In [126]:
for r in reddit_output:
    print(r)
    filename = 'scraper/' + r
    with open(filename) as f:
        c = json.load(f)
        send_reddit_json(c)

TheGirlSurvivalGuide.json
GirlsGoneWired.json
feminism.json
antifeminists.json
femmethoughts.json
GirlGamers.json
scientits.json
BreadTube.json
blackladies.json
NotHowGirlsWork.json
ContraPoints.json
FemaleDatingStrategy.json
women.json
WitchesVsPatriarchy.json


In [127]:
filename = 'Feminism_1_subreddit.json'
with open(filename) as f:
    c = json.load(f)
    send_reddit_json(c)

In [128]:
filename = 'feminism_subreddit.json'
with open(filename) as f:
    c = json.load(f)
    send_reddit_json(c)

In [129]:
filename = 'feminisms_subreddit.json'
with open(filename) as f:
    c = json.load(f)
    send_reddit_json(c)

In [130]:
filename = 'GenderCritical_subreddit.json'
with open(filename) as f:
    c = json.load(f)
    send_reddit_json(c)

In [131]:
filename = 'TwoXChromosomes_subreddit.json'
with open(filename) as f:
    c = json.load(f)
    send_reddit_json(c)

In [132]:
filename = 'TwoXChromosomes.json'
with open(filename) as f:
    c = json.load(f)
    send_reddit_json(c)

In [133]:
twitter_mapping = {
  "mappings": {
    "dynamic": False,
    "date_detection": False,
    "properties": {
      "id": {
        "type": "keyword"
      },
      "created_at": {
        "type": "date"
      },
      "text": {
        "type": "keyword"
      },
      "hashtags": {
        "type": "keyword"
      },
      "symbols": {
        "type": "keyword"
      },
      "user_id": {
        "type": "keyword"
      },
      "user_location": {
        "type": "keyword"
      },
      "geo": {
        "type": "keyword"
      },
      "coordinates": {
        "type": "keyword"
      },
      "retweet_count": {
        "type": "keyword"
      },
      "favorite_count": {
        "type": "keyword"
      },
      "possibly_sensitive": {
        "type": "keyword"
      }
    }
  }
}

In [134]:
response = es.indices.create(
    index="twitter",
    body=twitter_mapping,
    ignore=400 # ignore 400 already exists code
)

In [135]:
response

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'twitter'}

In [189]:
twitter_output = [dI for dI in os.listdir('scraper') if not os.path.isdir(os.path.join('scraper',dI)) \
                 and dI.startswith('twitter') and dI.endswith('json')]


In [190]:
twitter_output

['twitter_womenempowerment_tag.json',
 'twitter_effyourbeautystandards_tag.json',
 'twitter_woman_search.json',
 'twitter_nomoore_tag.json',
 'twitter_yesallwomen_tag.json',
 'twitter_woman_tag.json',
 'twitter_whyididntreport_tag.json',
 'twitter_iamanastywoman_tag.json',
 'twitter_metoo_tag.json',
 'twitter_timesup_tag.json',
 'twitter_equality_tag.json',
 'twitter_womensreality_tag.json',
 'twitter_whyistayed_tag.json',
 'twitter_gender equality_search.json',
 'twitter_iamwithher_tag.json',
 'twitter_fem2_tag.json',
 'twitter_feminist_search.json',
 'twitter_heforshe_tag.json',
 'twitter_feminist_tag.json',
 'twitter_everudaysexism_tag.json',
 'twitter_sexism_search.json',
 'twitter_tothegirls_tag.json',
 'twitter_girlpower_tag.json',
 'twitter_feminism_search.json',
 'twitter_women_tag.json']

In [191]:
def send_twitter_json(content):
    for r in content:
        date = int(time.mktime(datetime.datetime.strptime(r['created_at'], '%a %b %d %H:%M:%S +0000 %Y').timetuple()) * 1000)
        c = ' '.join(r['coordinates']) if r['coordinates'] is not None else ''
        doc = {
            'id': r['id_str'],
            'created_at': date,
            'text': r['text'],
            'hashtags': list(map(lambda d: d['text'], r['entities']['hashtags'])),
            'symbols': None,
            'user_id': r['user']['id_str'],
            'user_location': r['user']['location'],
            'geo': str(r['geo']),
            'coordinates': str(r['coordinates']),
            'retweet_count': r['retweet_count'],
            'favorite_count': r['favorite_count'],
            'possibly_sensitive': str(r.get('possibly_sensitive', False))
        }
        resp = es.index(index='twitter', body=doc, id=r['id'])

In [192]:
for t in twitter_output:
    filename = 'scraper/' + t
    print(filename)
    with open(filename) as f:
        c = json.load(f)
        send_twitter_json(c)

scraper/twitter_womenempowerment_tag.json
scraper/twitter_effyourbeautystandards_tag.json
scraper/twitter_woman_search.json
scraper/twitter_nomoore_tag.json
scraper/twitter_yesallwomen_tag.json
scraper/twitter_woman_tag.json
scraper/twitter_whyididntreport_tag.json
scraper/twitter_iamanastywoman_tag.json
scraper/twitter_metoo_tag.json
scraper/twitter_timesup_tag.json
scraper/twitter_equality_tag.json
scraper/twitter_womensreality_tag.json
scraper/twitter_whyistayed_tag.json
scraper/twitter_gender equality_search.json
scraper/twitter_iamwithher_tag.json
scraper/twitter_fem2_tag.json
scraper/twitter_feminist_search.json
scraper/twitter_heforshe_tag.json
scraper/twitter_feminist_tag.json
scraper/twitter_everudaysexism_tag.json
scraper/twitter_sexism_search.json
scraper/twitter_tothegirls_tag.json
scraper/twitter_girlpower_tag.json
scraper/twitter_feminism_search.json
scraper/twitter_women_tag.json
